# Experiment Notebook: Network Issuance and Inflation Rate

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis: Inflation Rate and ETH Supply Over Time](#Analysis:-Inflation-Rate-and-ETH-Supply-Over-Time)

# Experiment Summary 

The purpose of this notebook is to explore the ETH issuance and resulting annualized inflation rate across different time horizons and adoption scenarios. 

# Experiment Assumptions

See [assumptions document](ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
import setup

import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from model.types import Stage
from data.historical_values import df_ether_supply

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

In [ ]:
# Fetch the time-domain analysis experiment
experiment = time_domain_analysis.experiment
# Create a copy of the experiment simulation
simulation = copy.deepcopy(experiment.simulations[0])

In [ ]:
# Experiment configuration

simulation_names = {
    'Validator Adoption Scenarios': [
        'Normal Adoption',
        'Low Adoption',
        'High Adoption'
    ],
    'PoS Launch Date Scenarios': [
        "2021/12/1",
        "2022/03/1",
        "2022/06/1",
        "2022/09/1",
    ],
    'EIP1559 Scenarios': [
        'Disabled',
        'Enabled: Steady State',
        'Enabled: MEV',
    ]
}

simulation_1 = copy.deepcopy(simulation)
simulation_1.model.params.update({
    'validator_process': [
        lambda _run, _timestep: 3,  # Normal adoption: current average active validators per epoch from Beaconscan
        lambda _run, _timestep: 3 * 0.5,  # Low adoption: 50% lower scenario
        lambda _run, _timestep: 3 * 1.5,  # High adoption: 50% higher scenario
    ],
})

simulation_2 = copy.deepcopy(simulation)
simulation_2.model.params.update({
    'date_pos': [
        datetime.strptime("2021/12/1", "%Y/%m/%d"),
        datetime.strptime("2022/03/1", "%Y/%m/%d"),
        datetime.strptime("2022/06/1", "%Y/%m/%d"),
        datetime.strptime("2022/09/1", "%Y/%m/%d"),
    ],
})

simulation_3 = copy.deepcopy(simulation)
simulation_3.model.params.update({
    'eip1559_basefee_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 100, # Enabled: Steady state
        lambda _run, _timestep: 70, # Enabled: MEV
    ],  # Gwei per gas
    'eip1559_tip_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 1, # Enabled: Steady state
        lambda _run, _timestep: 30, # Enabled: MEV
    ],  # Gwei per gas
})

experiment.simulations = [
    simulation_1,
    simulation_2,
    simulation_3
]

In [ ]:
df, _exceptions = run(experiment)

In [ ]:
df = df.set_index('timestamp', drop=False)

# Analysis: Inflation Rate and ETH Supply Over Time

TODO: update description re. scenarios when finalized

This analysis allows the exploration of inflation rate and ETH supply over time, and supports the three adoption scenarios introduced in the second analysis notebook. The activation dates of the two major milestones in this analysis, EIP1559 and Proof of Stake, can be customized to simulate bespoke scenarios.

## WIP Visualisations

TODO: combine the following two visualizations, or create sliders for each parameter option

See WIP combination `plot_eth_supply_and_inflation_over_all_stages_wip(...)`

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()

initial_simulation = 1
for subset in df.query(f'simulation == {initial_simulation}').subset.unique():
    simulation_key = list(simulation_names.keys())[initial_simulation]
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df.query(f'subset == {subset} and simulation == 1').eth_supply,
            name=simulation_names[simulation_key][subset],
            visible=True
        )
    )

buttons = []

for simulation_index in df.simulation.unique():
    simulation_key = list(simulation_names.keys())[simulation_index]
    simulation_df = df.query(f'simulation == {simulation_index}')
    buttons.append(dict(method='update',
                        label=str(simulation_key),
                        visible=True,
                        args=[
                            {
                                'y': [
                                    simulation_df.query(f'subset == {subset}').eth_supply \
                                    for subset in simulation_df.subset.unique()
                                ],
                                'x':[df.index],
                                'name': list([
                                    simulation_names[simulation_key][subset] for subset in simulation_df.subset.unique()
                                ]),
                                'type':'scatter'
                            }, [subset for subset in simulation_df.subset.unique()]
                        ],
                    ))

fig.update_layout(updatemenus=[dict(
    type='buttons',
    buttons=buttons,
    direction='right',
    showactive=True,
    pad={"r": 10, "t": 10},
    x=0.5,
    xanchor="center",
    y=1.1,
    yanchor="top"
)])
fig.update_layout(hovermode='x unified')
fig.show()

In [ ]:
from radcad.utils import generate_cartesian_product_parameter_sweep

simulation_0 = copy.deepcopy(simulation)

param_sweep = generate_cartesian_product_parameter_sweep({
#     'validator_process': [
#         lambda _run, _timestep: 3,  # Normal adoption: current average active validators per epoch from Beaconscan
#         lambda _run, _timestep: 3 * 0.5,  # Low adoption: 50% lower scenario
#         lambda _run, _timestep: 3 * 1.5,  # High adoption: 50% higher scenario
#     ],
#     'date_pos': [
#         datetime.strptime("2021/12/1", "%Y/%m/%d"),
#         datetime.strptime("2022/12/1", "%Y/%m/%d"),
#         datetime.strptime("2023/12/1", "%Y/%m/%d"),
#         datetime.strptime("2024/12/1", "%Y/%m/%d"),
#     ],
    'eip1559_basefee_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 100, # Enabled: Steady state
        lambda _run, _timestep: 70, # Enabled: MEV
    ],  # Gwei per gas
    'eip1559_tip_process': [
        lambda _run, _timestep: 0, # Disabled
        lambda _run, _timestep: 1, # Enabled: Steady state
        lambda _run, _timestep: 30, # Enabled: MEV
    ],  # Gwei per gas
})

simulation_0.model.params.update(param_sweep)

In [ ]:
df_0, _exceptions = run(simulation_0)

In [ ]:
# TODO: Create a dropdown to choose between the three adoption scenarios
# TODO: Create a dropdown or slider to allow for the customization of EIP1559 and PoS dates
visualizations.plot_eth_supply_and_inflation_over_all_stages(df_ether_supply, df_0)